In [615]:
import pandas as pd

import random
import nltk
#nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from num2words import num2words

# Reading Data


In [616]:
df = pd.read_csv("/home/mayank/Desktop/JEOPARDY_CSV.csv",names =["shownum","airdate","round","category","value","ques","ans"],header= None).dropna()

/home/mayank/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [617]:
df.columns = [col.strip() for col in df.columns]
df.shape

(216929, 7)

In [618]:
df = df.iloc[1:]
df.reset_index()
#df = df.head(100000) # consider first 100k samples 


,index,shownum,airdate,round,category,value,ques,ans
0,1,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,2,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,3,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,5,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams
...,...,...,...,...,...,...,...,...
216923,216926,4999,2006-05-11,Double Jeopardy!,RIDDLE ME THIS,$2000,This Puccini opera turns on the solution to 3 ...,Turandot
216924,216927,4999,2006-05-11,Double Jeopardy!,"""T"" BIRDS",$2000,In North America this term is properly applied...,a titmouse
216925,216928,4999,2006-05-11,Double Jeopardy!,AUTHORS IN THEIR YOUTH,$2000,"In Penny Lane, where this ""Hellraiser"" grew up...",Clive Barker
216926,216929,4999,2006-05-11,Double Jeopardy!,QUOTATIONS,$2000,"From Ft. Sill, Okla. he made the plea, Arizona...",Geronimo


In [619]:
df = df.sample(frac=1).reset_index(drop=True) # reset index
df


,shownum,airdate,round,category,value,ques,ans
0,6250,2011-11-18,Jeopardy!,A LITTLE ALLITERATION,$800,This French phrase refers to a dangerously att...,femme fatale
1,3724,2000-11-09,Double Jeopardy!,A PASSION FOR FASHION,$200,"Worn today by both men & women, the coat varie...",Bomber jacket
2,3816,2001-03-19,Double Jeopardy!,AUTHORS,$200,"This ""Don Quixote"" author was called the ""Maim...",Cervantes
3,5731,2009-07-06,Double Jeopardy!,POET-TREE,$1200,"""The woods decay, the woods decay and fall... ...",Tennyson
4,5431,2008-03-31,Double Jeopardy!,"IS IT ""TEA"" TIME YET?",$1600,When Lady Diana had doubts about marrying Prin...,tea towels
...,...,...,...,...,...,...,...
216923,5201,2007-04-02,Double Jeopardy!,SPORTS 2006,$1600,Guard Deanna Nolan was the finals MVP as the D...,the WNBA
216924,2940,1997-05-16,Jeopardy!,PEOPLE,$500,This retired cartoonist was recently quoted as...,Gary Larson
216925,4678,2004-12-29,Jeopardy!,B.C. TIMES,$400,In 1998 Egypt unveiled the results of its 10-y...,the sphinx
216926,5516,2008-09-08,Jeopardy!,ZOOLOGY,$200,In the Amazon one species of this flying mamma...,a bat


In [620]:
# # Source: https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn
# class MultiColumnLabelEncoder:
#     def __init__(self,columns = None):
#         self.columns = columns # array of column names to encode

#     def fit(self,X,y=None):
#         return self # not relevant here

#     def transform(self,X):
#         '''
#         Transforms columns of X specified in self.columns using
#         LabelEncoder(). If no columns specified, transforms all
#         columns in X.
#         '''
#         output = X.copy()
#         if self.columns is not None:
#             for col in self.columns:
#                 output[col] = LabelEncoder().fit_transform(output[col])
#         else:
#             for colname,col in output.iteritems():
#                 output[colname] = LabelEncoder().fit_transform(col)
#         return output

#     def fit_transform(self,X,y=None):
#         return self.fit(X,y).transform(X)

In [621]:
# df = MultiColumnLabelEncoder(columns = ['value']).fit_transform(df)


In [622]:
# s = set()
# for i in range(len(df)):
#     s.add(df['category'][i])
# # Values have total 86 Different types of entires or dollors values 
# print(len(s))

# # There are total of 3429 different catagories


In [623]:
#Not considering all values:
# d = {"$200": 0,"$1000": 1}

# for i in range(len(df)):
# #     print(type(d[df["value"][i]]))
# #     break
#     if df['value'][i] in d:
#         df['value'][i] = d[df['value'][i]]
#     else:
#         df['value'][i] = 'None'
# df     




In [624]:
# df = df.sample(frac=1).reset_index(drop=True)
# df = df[df['value'] != 'None']


In [625]:
# Different prices in the dataset
from collections import Counter
print(Counter(df["value"]))

Counter({'$400': 42243, '$800': 31860, '$200': 30454, '$600': 20377, '$1000': 19539, '$1200': 11331, '$2000': 11243, '$1600': 10801, '$100': 9029, '$500': 9016, '$300': 8663, 'None': 3634, '$1,000': 2101, '$2,000': 1586, '$3,000': 769, '$1,500': 546, '$1,200': 441, '$4,000': 349, '$1,600': 239, '$2,500': 232, '$5,000': 231, '$1,400': 228, '$700': 203, '$1,800': 182, '$2,200': 147, '$2,400': 127, '$900': 114, '$6,000': 85, '$2,600': 83, '$1,300': 75, '$2,800': 69, '$1,100': 63, '$3,600': 56, '$3,200': 53, '$3,400': 51, '$3,500': 49, '$1,700': 44, '$4,200': 43, '$7,000': 39, '$3,800': 37, '$1,900': 28, '$5,200': 25, '$4,800': 24, '$2,300': 23, '$10,000': 22, '$2,100': 22, '$4,600': 21, '$8,000': 20, '$2,700': 18, '$4,500': 16, '$4,400': 15, '$5,600': 13, '$5,400': 12, '$3,100': 11, '$6,400': 11, '$5,800': 10, '$3,300': 9, '$5': 9, '$7,800': 8, '$9,000': 8, '$6,600': 8, '$6,200': 7, '$3,700': 7, '$5,500': 7, '$10,800': 6, '$2,900': 6, '$250': 5, '$7,200': 5, '$3,900': 5, '$8,200': 4, '$7,

# Data Cleaning


In [626]:
import re

def clean_str(string):
    """Tokenization/string cleaning for all datasets except for SST.

    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"[.,#!$%&;:{}=_`~()/\\]", "", string)
    return string.strip().lower()



df['category'] = df['category'].astype(str).map(clean_str,na_action=None)
df['ques'] = df['ques'].astype(str).map(clean_str,na_action=None)
df['ans'] = df['ans'].astype(str).map(clean_str,na_action=None)


# Stemming and Lemmatization

from nltk.stem.porter import PorterStemmer

def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

df['category'] = get_stemmed_text(df['category'])
df['ques'] = get_stemmed_text(df['ques'])
df['ans'] = get_stemmed_text(df['ans'])

from nltk.stem import WordNetLemmatizer
def get_lemmatized_text(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

df['category'] = get_lemmatized_text(df['category'])
df['ques'] = get_lemmatized_text(df['ques'])
df['ans'] = get_lemmatized_text(df['ans'])

# Convert numbers to letters

def convert_num_to_words(utterance):
      utterance = ' '.join([num2words(i) if i.isdigit() else i for i in utterance.split()])
      return utterance


df["category"] = df["category"].apply(lambda text: convert_num_to_words(str(text)))
df["ans"] = df["ans"].apply(lambda text: convert_num_to_words(str(text)))
df["ques"] = df["ques"].apply(lambda text: convert_num_to_words(str(text)))

df = df.sample(frac=1).reset_index(drop=True) # reset index
df

,shownum,airdate,round,category,value,ques,ans
0,5542,2008-10-14,Jeopardy!,found in translat,$200,liviu andronicu start latin epic poetri by tra...,homer
1,3728,2000-11-15,Jeopardy!,see attl,$400,known a the emp for short thi music museum see...,experi music project
2,2918,1997-04-16,Jeopardy!,act of the apostl,$300,the apostl believ judas' act of betray wa pred...,david
3,4996,2006-05-08,Double Jeopardy!,a world of fact,$1200,thi capit of madhya pradesh state wa the site ...,bhopal
4,3285,1998-12-11,Double Jeopardy!,the oscar,$200,he 's the first man sinc walter brennan to win...,jack nicholson
...,...,...,...,...,...,...,...
216923,3669,2000-07-13,Double Jeopardy!,famou for fifteen minut,$400,a categori on famou houseguest would have to i...,kato kaelin
216924,4819,2005-07-14,Double Jeopardy!,song hit for two,$1600,"sheryl crow in one thousand, nine hundred and ...",sweet child o' mine
216925,4633,2004-10-27,Double Jeopardy!,potent potabl,$1600,cheer to thi babylonian king who codifi the us...,hammurabi
216926,4922,2006-01-24,Double Jeopardy!,six foot under,$1200,at 5'4 tall a href http www j archiv com mediu...,jame madison


In [627]:
del df['shownum']  # deleting shownum and airdate rows as they are not important features
del df['airdate']


In [628]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,round,category,value,ques,ans
0,Jeopardy!,gener scienc,"$1,000",titan a moon of thi planet ha a thicker atmosp...,saturn
1,Double Jeopardy!,feelin' gouti,$1000,thi author wa crippl by gout blifil 's mother ...,henri field
2,Double Jeopardy!,garden,$200,if your soil is too heavi or sandi you can add...,peat moss
3,Jeopardy!,lesser known music,$400,bulli for len cariou who play thi famou man in...,theodor roosevelt
4,Final Jeopardy!,american author,None,"in one thousand, nine hundred and fifty-eight ...",truman capot


# Data Filtering

In [629]:
df = df[df['round'] == 'Jeopardy!']  # select only Jeopardy round data
#print(df.shape)
df = df[df.ans.str.isalpha() == True]  # selecting only those rows in which ans is a string

df.shape


(44986, 5)

In [630]:
# Can also delete round too as it is constant now because we are only considering Jeopardy round
del df['round']

In [631]:
df = df[~df.ques.str.contains("<a href=")]
df.shape

(44986, 4)

In [632]:
# Filter questions where len of ques is equal or more than 5
df = df[df['ques'].str.split().str.len() >= 5]
df.shape

(41857, 4)

In [633]:
df = df.sample(frac=1).reset_index(drop=True)
df

,category,value,ques,ans
0,what are you made of ?,$800,st john the baptist by donatello at siena cathedr,bronz
1,tv movi tourism,$200,hossburg are serv at the ponderosa ranch in in...,bonanza
2,medicin plant,$300,span missionari in peru found a substanc in th...,malaria
3,commun,$300,funk wagnal defin it a to draw the eyelid of o...,wink
4,d brief,$800,"until one thousand, nine hundred and seventy-o...",defer
...,...,...,...,...
41852,hamnesia,$600,ok i'm on some kind of farm what doe that say ...,babe
41853,have a beer a slogan,$200,brew with pure rocki mountain spring water,coor
41854,end in ett,$400,howdi doodi wa one of these so you had to pull...,marionett
41855,potluck,$600,the name of thi dark brownish red come from th...,maroon


In [636]:
# Final Data Prepraration
# Class Balanced Data with 50% samples with $200 and vice-versa
df_200 = df[df['value']=='$200'].sample(2000, random_state=670)
df_1000 = df[df['value']=='$1000'].sample(2000, random_state=670)
df_400 = df[df['value']=='$400'].sample(2000, random_state=670)
df_800 = df[df['value']=='$800'].sample(2000, random_state=670)
df_600 = df[df['value']=='$600'].sample(2000, random_state=670)

df = pd.concat([df_200,df_1000,df_400,df_800,df_600])
df = df.sample(frac=1).reset_index(drop=True)

df

,category,value,ques,ans
0,have a whiskey,$600,the world 's most popular whiskey is thi color...,red
1,x rate,$800,thi great persian king rule from approxim four...,xerx
2,sound all around,$1000,thi rude nois is the last name of tobi a chara...,belch
3,countri by newspap,$200,el financiero the oaxaca time,mexico
4,home from the hous,$400,speaker boehner goe home to thi state where he...,ohio
...,...,...,...,...
9995,music,$800,now a one act music on it own the lorax wa ori...,seussic
9996,travel tourism,$200,you may wander past wallabi when you bushwalk ...,australia
9997,water tast off,$600,pure yet piquant it must be thi from a spa tow...,evian
9998,world capit,$200,the oldest part of thi northern european capit...,stockholm


In [637]:
# Saving the modified data, will be used later 
df.to_csv('/home/mayank/Desktop/mod_data.csv')

In [646]:
# Loading the new data
df = pd.read_csv("/home/mayank/Desktop/mod_data.csv")
df

,Unnamed: 0,category,value,ques,ans
0,0,have a whiskey,$600,the world 's most popular whiskey is thi color...,red
1,1,x rate,$800,thi great persian king rule from approxim four...,xerx
2,2,sound all around,$1000,thi rude nois is the last name of tobi a chara...,belch
3,3,countri by newspap,$200,el financiero the oaxaca time,mexico
4,4,home from the hous,$400,speaker boehner goe home to thi state where he...,ohio
...,...,...,...,...,...
9995,9995,music,$800,now a one act music on it own the lorax wa ori...,seussic
9996,9996,travel tourism,$200,you may wander past wallabi when you bushwalk ...,australia
9997,9997,water tast off,$600,pure yet piquant it must be thi from a spa tow...,evian
9998,9998,world capit,$200,the oldest part of thi northern european capit...,stockholm


# Finetuning Fastext model on pretrained wikipedia news model

In [647]:
# Generate fasttext model for "ques","ans","category" col separately

df_new = df
del df_new["category"]
del df_new["ans"]
df_new




,Unnamed: 0,value,ques
0,0,$600,the world 's most popular whiskey is thi color...
1,1,$800,thi great persian king rule from approxim four...
2,2,$1000,thi rude nois is the last name of tobi a chara...
3,3,$200,el financiero the oaxaca time
4,4,$400,speaker boehner goe home to thi state where he...
...,...,...,...
9995,9995,$800,now a one act music on it own the lorax wa ori...
9996,9996,$200,you may wander past wallabi when you bushwalk ...
9997,9997,$600,pure yet piquant it must be thi from a spa tow...
9998,9998,$200,the oldest part of thi northern european capit...


In [648]:
# Fast Text 
with open("/home/mayank/Desktop/ques.txt", "w") as train_file_handler:
    for X_train_entry, y_train_entry in zip(df_new['ques'],df_new['value']):
        line_to_write = "__label__" + str(y_train_entry) + "\t" + str(X_train_entry) + "\n"
        try:
            train_file_handler.write(line_to_write)
        except:
            print(line_to_write)
            break
        



In [649]:
# These fastext model will be used later
import re
import fasttext
import numpy as np
import pandas as pd
from fasttext import load_model

hyper_params = { 
    "lr": 0.001,         # Learning rate
    "epoch": 25,       # Number of training epochs to train for
    "wordNgrams": 4,    # Number of word n-grams to consider during training
    "dim": 300,         # Size of word vectors
    "ws": 5,        # Size of the context window for CBOW or skip-gram
    'lrUpdateRate': 15000,
    'bucket' : 100000,
    "loss" : 'ns',
    "maxn" : 5,
    "minn" : 2,
    "minCount":1
    }
print("Training Started")
model = fasttext.train_supervised(input='/home/mayank/Desktop/ques.txt',**hyper_params,verbose = 10,pretrainedVectors='/home/mayank/Downloads/pmeans/wiki-news-300d-1M.vec')
print("Training Ended")



print("Saving the model...........")
model.save_model('/home/mayank/Desktop/ques_model.bin')
print("Model saved successfully")

Training Started
Training Ended
Saving the model...........
Model saved successfully
